In [3]:
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.types import IntegerType, StringType, DateType
from datetime import datetime as dt, timedelta
from pyspark.sql import functions as F
from bs4 import BeautifulSoup
from app_s3 import S3 # LIB AWS
from datetime import datetime as dt
from dotenv import load_dotenv # QUANDO NOS TIVERMOS O COFRE DE SEMHAS TROCAREMOS ESSA LIB PELA LIB DE CONSUMO DE CHAVES DA AWS
import pandas as pd
import requests
import boto3
import io
import os


In [12]:
import boto3
import pandas as pd
import io
import os
from dotenv import load_dotenv

class S3:
    def __init__(self):
        # Carregar as credenciais do arquivo .env
        dotenv_path = os.path.join('.env')
        load_dotenv(dotenv_path)

        # Credenciais AWS
        self.pbi_access_key_id = os.getenv("pbi_access_key_id")
        self.pbi_secret_access_key = os.getenv("pbi_secret_access_key")

    # Método para ler arquivo CSV do S3
    def ler_csv_bucket_bruto(self, nm_arq):
        bucket = "bi-storage-labutare"
        key = f"PRD/Bronze/{nm_arq}/{nm_arq}.csv"

        # Criação do cliente S3 com boto3
        s3 = boto3.client(
            's3',
            aws_access_key_id=self.pbi_access_key_id,
            aws_secret_access_key=self.pbi_secret_access_key,
            region_name='us-east-1'
        )

        # Baixando o arquivo do S3
        obj = s3.get_object(Bucket=bucket, Key=key)
        csv_string = obj['Body'].read().decode('utf-8')

        # Usando pandas para ler o CSV a partir da string
        df = pd.read_csv(io.StringIO(csv_string), sep=',')
        return df


# Exemplo de uso da classe S3
s3 = S3()
df_enterprises = s3.ler_csv_bucket_bruto("enterprises")

# Realizando as transformações no Pandas
df_enterprises['id'] = df_enterprises['id'].astype(int)
df_enterprises['nome'] = df_enterprises['name'].astype(str)
df_enterprises['nome_comercial'] = df_enterprises['commercialName'].astype(str)
df_enterprises['cnpj'] = df_enterprises['cnpj'].astype(str)
df_enterprises['tipo'] = df_enterprises['type'].astype(int)
df_enterprises['endereco'] = df_enterprises['adress'].astype(str)
df_enterprises['data_criacao'] = pd.to_datetime(df_enterprises['creationDate'])
df_enterprises['data_modificacao'] = pd.to_datetime(df_enterprises['modificationDate'])
df_enterprises['criado_por'] = df_enterprises['createdBy'].astype(str)
df_enterprises['modificado_por'] = df_enterprises['modifiedBy'].astype(str)
df_enterprises['empresa_id'] = df_enterprises['companyId'].astype(int)
df_enterprises['empresa_nome'] = df_enterprises['companyName'].astype(str)
df_enterprises['banco_custos_id'] = df_enterprises['costDatabaseId'].astype(str)
df_enterprises['banco_custos_descricao'] = df_enterprises['costDatabaseDescription'].astype(str)
df_enterprises['tipo_construcao_id'] = df_enterprises['buildingTypeId'].astype(int)
df_enterprises['tipo_construcao_descricao'] = df_enterprises['buildingTypeDescription'].astype(str)
df_enterprises['observacao_empr'] = df_enterprises['enterpriseObservation'].astype(str)

# Criar a coluna cnpj_numerico sem caracteres especiais
df_enterprises['cnpj_numerico'] = df_enterprises['cnpj'].str.replace(r'\D', '', regex=True)

# Exibir o conteúdo do DataFrame
#print(df_enterprises.head())



display(df_enterprises)

s3.salvar_na_silver(df_enterprises, "enterprises")

,id,name,commercialName,cnpj,type,adress,creationDate,modificationDate,createdBy,modifiedBy,...,criado_por,modificado_por,empresa_id,empresa_nome,banco_custos_id,banco_custos_descricao,tipo_construcao_id,tipo_construcao_descricao,observacao_empr,cnpj_numerico
0,101,ADMINISTRATIVO LABUTARE,NaN,nan,1,Campinas - SP,2024-09-04,2024-09-04,CHARLEY.CORNACHIONE,CHARLEY.CORNACHIONE,...,CHARLEY.CORNACHIONE,CHARLEY.CORNACHIONE,1,LABUTARE CONSTRUTORA E INCORPORADORA LTDA,1,Tabela geral,1,Construção Civil,nan,
1,102,FLUXO FINANCEIRO LABUTARE,NaN,nan,1,Campinas - SP,2024-09-04,2024-09-04,CHARLEY.CORNACHIONE,CHARLEY.CORNACHIONE,...,CHARLEY.CORNACHIONE,CHARLEY.CORNACHIONE,1,LABUTARE CONSTRUTORA E INCORPORADORA LTDA,1,Tabela geral,1,Construção Civil,nan,
2,103,COLISEUM - FRANCO DA ROCHA,LABUTARE - COLISEUM,nan,1,Franco da Rocha - SP,2024-09-04,2024-09-09,CHARLEY.CORNACHIONE,AUREA.ANDRADE,...,CHARLEY.CORNACHIONE,AUREA.ANDRADE,1,LABUTARE CONSTRUTORA E INCORPORADORA LTDA,1,Tabela geral,1,Construção Civil,nan,
3,104,ALAMEDAS - FRANCO DA ROCHA,LABUTARE - ALAMEDAS,nan,1,Franco da Rocha - SP,2024-09-09,2024-09-09,AUREA.ANDRADE,AUREA.ANDRADE,...,AUREA.ANDRADE,AUREA.ANDRADE,1,LABUTARE CONSTRUTORA E INCORPORADORA LTDA,1,Tabela geral,1,Construção Civil,nan,
4,105,BABILÔNIA - FRANCO DA ROCHA,LABUTARE - BABILÔNIA,nan,1,Franco da Rocha - SP,2024-09-09,2024-09-25,AUREA.ANDRADE,AUREA.ANDRADE,...,AUREA.ANDRADE,AUREA.ANDRADE,1,LABUTARE CONSTRUTORA E INCORPORADORA LTDA,1,Tabela geral,1,Construção Civil,nan,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1043,TANAKA - MOGI DAS CRUZES - LABUTARE CONCILIAÇÃO,NaN,nan,1,Mogi das Cruzes - SP,2024-11-14,2024-11-14,LUIZA.CARLLA,LUIZA.CARLLA,...,LUIZA.CARLLA,LUIZA.CARLLA,100,LABUTARE CONCILIAÇÃO,1,Tabela geral,1,Construção Civil,nan,
96,1044,BRAUNA - MOGI DAS CRUZES - SPE - LABUTARE CONC...,NaN,nan,1,Mogi das Cruzes - SP,2024-11-14,2024-11-14,LUIZA.CARLLA,LUIZA.CARLLA,...,LUIZA.CARLLA,LUIZA.CARLLA,100,LABUTARE CONCILIAÇÃO,1,Tabela geral,1,Construção Civil,nan,
97,1045,KOMURA SPE - MOGI DASD CRUZES - LABUTARE CONCI...,NaN,nan,1,Mogi das Cruzes - SP,2024-11-14,2024-11-14,LUIZA.CARLLA,LUIZA.CARLLA,...,LUIZA.CARLLA,LUIZA.CARLLA,100,LABUTARE CONCILIAÇÃO,1,Tabela geral,1,Construção Civil,nan,
98,1046,NOVA VALINHOS - LABUTARE CONCILIAÇÃO,NaN,nan,1,Valinhos - SP,2024-11-14,2024-11-14,LUIZA.CARLLA,LUIZA.CARLLA,...,LUIZA.CARLLA,LUIZA.CARLLA,100,LABUTARE CONCILIAÇÃO,1,Tabela geral,1,Construção Civil,nan,


AttributeError: 'S3' object has no attribute 'salvar_na_silver'

In [ ]:
import boto3
import pandas as pd
import io
import os
from dotenv import load_dotenv

class S3:
    def __init__(self):
        # Carregar as credenciais do arquivo .env
        dotenv_path = os.path.join('.env')
        load_dotenv(dotenv_path)

        # Credenciais AWS para leitura (usuário Power BI)
        self.pbi_access_key_id = os.getenv("pbi_access_key_id")
        self.pbi_secret_access_key = os.getenv("pbi_secret_access_key")

    # Método para ler arquivo CSV do S3
    def ler_csv_bucket_bruto(self, nm_arq):
        bucket = "bi-storage-labutare"
        key = f"PRD/Bronze/{nm_arq}/{nm_arq}.csv"

        # Criação do cliente S3 com credenciais de leitura (Power BI)
        s3 = boto3.client(
            's3',
            aws_access_key_id=self.pbi_access_key_id,
            aws_secret_access_key=self.pbi_secret_access_key,
            region_name='us-east-1'
        )

        # Baixando o arquivo do S3
        obj = s3.get_object(Bucket=bucket, Key=key)
        csv_string = obj['Body'].read().decode('utf-8')

        # Usando pandas para ler o CSV a partir da string
        df = pd.read_csv(io.StringIO(csv_string), sep=',')
        return df
    
    # Método para salvar na Silver
    def salvar_na_silver(self, df, nm_arq):
        bucket = "bi-storage-labutare"
        key_csv = f"PRD/Silver/{nm_arq}/{nm_arq}.csv"
        key_parquet = f"PRD/Silver/{nm_arq}/{nm_arq}.parquet"

        # Carregar credenciais específicas para escrita na Silver
        silver_access_key_id = os.getenv("silver_access_key_id")
        silver_secret_access_key = os.getenv("silver_secret_access_key")

        # Criar cliente S3 com credenciais de escrita
        s3 = boto3.client(
            's3',
            aws_access_key_id=silver_access_key_id,
            aws_secret_access_key=silver_secret_access_key,
            region_name='us-east-1'
        )

        # Salvar CSV
        csv_buffer = io.StringIO()
        df.to_csv(csv_buffer, index=False)
        s3.put_object(Bucket=bucket, Key=key_csv, Body=csv_buffer.getvalue())

        # Salvar Parquet
        parquet_buffer = io.BytesIO()
        df.to_parquet(parquet_buffer, index=False)
        s3.put_object(Bucket=bucket, Key=key_parquet, Body=parquet_buffer.getvalue())

# Exemplo de uso da classe S3
s3 = S3()
df_enterprises = s3.ler_csv_bucket_bruto("enterprises")

# Realizando as transformações no Pandas
df_enterprises['id'] = df_enterprises['id'].astype(int)
df_enterprises['nome'] = df_enterprises['name'].astype(str)
df_enterprises['nome_comercial'] = df_enterprises['commercialName'].astype(str)
df_enterprises['cnpj'] = df_enterprises['cnpj'].astype(str)
df_enterprises['tipo'] = df_enterprises['type'].astype(int)
df_enterprises['endereco'] = df_enterprises['adress'].astype(str)
df_enterprises['data_criacao'] = pd.to_datetime(df_enterprises['creationDate'])
df_enterprises['data_modificacao'] = pd.to_datetime(df_enterprises['modificationDate'])
df_enterprises['criado_por'] = df_enterprises['createdBy'].astype(str)
df_enterprises['modificado_por'] = df_enterprises['modifiedBy'].astype(str)
df_enterprises['empresa_id'] = df_enterprises['companyId'].astype(int)
df_enterprises['empresa_nome'] = df_enterprises['companyName'].astype(str)
df_enterprises['banco_custos_id'] = df_enterprises['costDatabaseId'].astype(str)
df_enterprises['banco_custos_descricao'] = df_enterprises['costDatabaseDescription'].astype(str)
df_enterprises['tipo_construcao_id'] = df_enterprises['buildingTypeId'].astype(int)
df_enterprises['tipo_construcao_descricao'] = df_enterprises['buildingTypeDescription'].astype(str)
df_enterprises['observacao_empr'] = df_enterprises['enterpriseObservation'].astype(str)

# Criar a coluna cnpj_numerico sem caracteres especiais
#df_enterprises['cnpj_numerico'] = df_enterprises['cnpj'].str.replace(r'\D', '', regex=True)

# Salvar na Silver
s3.salvar_na_silver(df_enterprises, "enterprises")

# Exibir o conteúdo do DataFrame
#display(df_enterprises)


,id,name,commercialName,cnpj,type,adress,creationDate,modificationDate,createdBy,modifiedBy,...,data_modificacao,criado_por,modificado_por,empresa_id,empresa_nome,banco_custos_id,banco_custos_descricao,tipo_construcao_id,tipo_construcao_descricao,observacao_empr
0,101,ADMINISTRATIVO LABUTARE,NaN,nan,1,Campinas - SP,2024-09-04,2024-09-04,CHARLEY.CORNACHIONE,CHARLEY.CORNACHIONE,...,2024-09-04,CHARLEY.CORNACHIONE,CHARLEY.CORNACHIONE,1,LABUTARE CONSTRUTORA E INCORPORADORA LTDA,1,Tabela geral,1,Construção Civil,nan
1,102,FLUXO FINANCEIRO LABUTARE,NaN,nan,1,Campinas - SP,2024-09-04,2024-09-04,CHARLEY.CORNACHIONE,CHARLEY.CORNACHIONE,...,2024-09-04,CHARLEY.CORNACHIONE,CHARLEY.CORNACHIONE,1,LABUTARE CONSTRUTORA E INCORPORADORA LTDA,1,Tabela geral,1,Construção Civil,nan
2,103,COLISEUM - FRANCO DA ROCHA,LABUTARE - COLISEUM,nan,1,Franco da Rocha - SP,2024-09-04,2024-09-09,CHARLEY.CORNACHIONE,AUREA.ANDRADE,...,2024-09-09,CHARLEY.CORNACHIONE,AUREA.ANDRADE,1,LABUTARE CONSTRUTORA E INCORPORADORA LTDA,1,Tabela geral,1,Construção Civil,nan
3,104,ALAMEDAS - FRANCO DA ROCHA,LABUTARE - ALAMEDAS,nan,1,Franco da Rocha - SP,2024-09-09,2024-09-09,AUREA.ANDRADE,AUREA.ANDRADE,...,2024-09-09,AUREA.ANDRADE,AUREA.ANDRADE,1,LABUTARE CONSTRUTORA E INCORPORADORA LTDA,1,Tabela geral,1,Construção Civil,nan
4,105,BABILÔNIA - FRANCO DA ROCHA,LABUTARE - BABILÔNIA,nan,1,Franco da Rocha - SP,2024-09-09,2024-09-25,AUREA.ANDRADE,AUREA.ANDRADE,...,2024-09-25,AUREA.ANDRADE,AUREA.ANDRADE,1,LABUTARE CONSTRUTORA E INCORPORADORA LTDA,1,Tabela geral,1,Construção Civil,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1043,TANAKA - MOGI DAS CRUZES - LABUTARE CONCILIAÇÃO,NaN,nan,1,Mogi das Cruzes - SP,2024-11-14,2024-11-14,LUIZA.CARLLA,LUIZA.CARLLA,...,2024-11-14,LUIZA.CARLLA,LUIZA.CARLLA,100,LABUTARE CONCILIAÇÃO,1,Tabela geral,1,Construção Civil,nan
96,1044,BRAUNA - MOGI DAS CRUZES - SPE - LABUTARE CONC...,NaN,nan,1,Mogi das Cruzes - SP,2024-11-14,2024-11-14,LUIZA.CARLLA,LUIZA.CARLLA,...,2024-11-14,LUIZA.CARLLA,LUIZA.CARLLA,100,LABUTARE CONCILIAÇÃO,1,Tabela geral,1,Construção Civil,nan
97,1045,KOMURA SPE - MOGI DASD CRUZES - LABUTARE CONCI...,NaN,nan,1,Mogi das Cruzes - SP,2024-11-14,2024-11-14,LUIZA.CARLLA,LUIZA.CARLLA,...,2024-11-14,LUIZA.CARLLA,LUIZA.CARLLA,100,LABUTARE CONCILIAÇÃO,1,Tabela geral,1,Construção Civil,nan
98,1046,NOVA VALINHOS - LABUTARE CONCILIAÇÃO,NaN,nan,1,Valinhos - SP,2024-11-14,2024-11-14,LUIZA.CARLLA,LUIZA.CARLLA,...,2024-11-14,LUIZA.CARLLA,LUIZA.CARLLA,100,LABUTARE CONCILIAÇÃO,1,Tabela geral,1,Construção Civil,nan
